# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

T. Henning  ->  T. Henning  |  ['T. Henning']
M. Flock  ->  M. Flock  |  ['M. Flock']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
B. Saha  ->  B. Saha  |  ['B. Saha']


T. Henning  ->  T. Henning  |  ['T. Henning']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
Arxiv has 64 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2508.03814


extracting tarball to tmp_2508.03814...

 done.


T. Henning  ->  T. Henning  |  ['T. Henning']


...

 done.


Found 80 bibliographic references in tmp_2508.04254/aa53124-24.bbl.
Issues with the citations
syntax error in line 424: '=' expected
Retrieving document from  https://arxiv.org/e-print/2508.04411


extracting tarball to tmp_2508.04411... done.
Retrieving document from  https://arxiv.org/e-print/2508.04692


extracting tarball to tmp_2508.04692... done.


Found 95 bibliographic references in tmp_2508.04692/aa55862-25.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.03814-b31b1b.svg)](https://arxiv.org/abs/2508.03814) | **Worlds Next Door: A Candidate Giant Planet Imaged in the Habitable Zone of $α$ Cen A. I. Observations, Orbital and Physical Properties, and Exozodi Upper Limits**  |
|| C. Beichman, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-08-07*|
|*Comments*| *Accepted to ApJL. 34 pages, 22 figures, 10 tables*|
|**Abstract**|            We report on coronagraphic observations of the nearest solar-type star, $\alpha$ Cen A, using the MIRI instrument on the James Webb Space Telescope. With three epochs of observation (August 2024, February 2025, and April 2025), we achieve a sensitivity sufficient to detect $T_{\rm eff}\approx$ 225-250 K (1-1.2 $R_{\rm Jup}$) planets between 1"-2" and exozodiacal dust emission at the level of $>$5-8$\times$ the brightness of our own zodiacal cloud. The lack of exozodiacal dust emission sets an unprecedented limit of a few times the brightness of our own zodiacal cloud$-$a factor of $\gtrsim$10 more sensitive than measured toward any other stellar system to date. In August 2024, we detected a F$_\nu$(15.5 $\mu$m) = 3.5 mJy point source, called $S1$, at a separation of 1.5" from $\alpha$ Cen A. Because the August 2024 epoch had only one successful observation at a single roll angle, it is not possible to unambiguously confirm $S1$ as a bona fide planet. Our analysis confirms that $S1$ is neither a background nor a foreground object. $S1$ is not recovered in the February and April 2025 epochs. However, if $S1$ is the counterpart of the object, $C1$, seen by the VLT/NEAR program in 2019, we find that there is a 52% chance that the $S1+C1$ candidate was missed in both follow-up JWST/MIRI observations due to orbital motion. Incorporating constraints from the non-detections, we obtain families of dynamically stable orbits for $S1+C1$ with periods between 2-3 years. These suggest that the planet candidate is on an eccentric ($e \approx 0.4$) orbit significantly inclined with respect to $\alpha$ Cen AB orbital plane ($i_{\rm mutual} \approx 50^\circ$, or $\approx 130^\circ$). Based on the photometry and orbital properties, the planet candidate could have a temperature of 225 K, a radius of $\approx$1-1.1 $R_{\rm Jup}$ and a mass between 90-150 $M_{\rm Earth}$, consistent with RV limits.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.04254-b31b1b.svg)](https://arxiv.org/abs/2508.04254) | **Multi dust species inner rim in magnetized protoplanetary disks**  |
|| <mark>M. Flock</mark>, et al. -- incl., <mark>M. Benisty</mark>, <mark>R. v. Boekel</mark> |
|*Appeared on*| *2025-08-07*|
|*Comments*| *accepted at A&A*|
|**Abstract**|            The inner regions of protoplanetary disks, within ten astronomical units, are where terrestrial planets are born. By developing a new class of multi-dust radiative magnetized inner rim models, we can gain valuable insights into the conditions during planet formation. Our goal is twofold: to study the influence of highly refractory dust species on the inner rim shape and to determine how the magnetic field affects the inner disk structure. The resulting temperature and density structures are analyzed and compared to observations. The comparison focuses on a median SED of Herbig stars and interferometric constraints from the H, K, and N-band of three Herbig-type star-disk systems: HD 100546, HD 163296, and HD 169142. We investigate 1) the influence of a large-scale magnetic field on the inner disk structure and 2) the effect of having the four most important dust species (corundum, iron, forsterite, and enstatite) shaping the rim. We use frequency-dependent irradiation and the effect of accretion heating. With the Optool package, we obtain frequency-dependent opacities for each dust grain family and calculate the corresponding temperature-dependent Planck and Rosseland opacities. Our models with multiple dust species show a smoother and radially more extended inner rim. Strongly magnetized disks show a substantial increase in the emission flux between the L and N-bands. Weakly magnetized disk models with large-scale vertical magnetic fields < 0.3 Gauss at 1 au best fit with NIR interferometric observations. Our model comparison supports the existence of moderate magnetic fields ($\beta$ > $10^4$), which could still drive a magnetic wind in the inner disk. Our results show that multi-dust models, including magnetic fields, still lack NIR emission, especially in the H-band. One potential solution might be a heated gas disk or evaporating objects like planetesimals close to the star.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.04692-b31b1b.svg)](https://arxiv.org/abs/2508.04692) | **A transition from H$_2$O to C$_2$H$_2$ dominated spectra with decreasing stellar luminosity**  |
|| S. L. Grant, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark> |
|*Appeared on*| *2025-08-07*|
|*Comments*| *Accepted for publication in A&A. 15 pages, 9 figures. ArXiv abstract is shortened*|
|**Abstract**|            The chemical composition of the inner regions of disks around young stars will determine the properties of planets forming there. Many disk physical processes drive the chemical evolution, some of which depend on/correlate with the stellar properties. We aim to explore the connection between stellar properties and inner disk chemistry, using mid-infrared spectroscopy. We use JWST-MIRI observations of a large, diverse sample of sources to explore trends between C$_2$H$_2$ and H$_2$O. Additionally, we calculate the average spectrum for the T Tauri ($M_{*}$$>$0.2 $M_{\odot}$) and very low-mass star (VLMS, $M_{*}$$\leq$0.2 $M_{\odot}$) samples and use slab models to determine the properties. We find a significant anti-correlation between the flux ratio of C$_2$H$_2$/H$_2$O and the stellar luminosity. Disks around VLMS have significantly higher $F_{\rm{C_2H_2}}$/$F_{\rm{H_2O}}$ flux ratios than their higher-mass counterparts. We also explore trends with the strength of the 10 $\mu$m silicate feature, stellar accretion rate, and disk dust mass, all of which show correlations with the flux ratio, which may be related to processes driving the carbon-enrichment in disks around VLMS, but also have degeneracies with system properties. Slab model fits to the average spectra show that the VLMS H$_2$O emission is quite similar in temperature and column density to a warm ($\sim$600 K) H$_2$O component in the T Tauri spectrum, indicating that the high C/O gas phase ratio in these disks is not due to oxygen depletion alone. Instead, the presence of many hydrocarbons, including some with high column densities, points to carbon enhancement in the disks around VLMS. The observed differences in the inner disk chemistry as a function of host properties are likely to be accounted for by differences in the disk temperatures, stellar radiation field, and the evolution of dust grains.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.04411-b31b1b.svg)](https://arxiv.org/abs/2508.04411) | **Can fractal dimension distinguish between grand-design and flocculent spiral arms?**  |
|| <mark>B. Saha</mark>, S. Dutta, A. Banerjee |
|*Appeared on*| *2025-08-07*|
|*Comments*| *Accepted for publication in The Astrophysical Journal*|
|**Abstract**|            About two-thirds of disk galaxies host spiral arms, ranging from well-delineated grand-design spirals to fragmented flocculent spiral galaxies. We introduce fractal dimension $D_B$ as a non-parametric measure to distinguish between grand-designs and flocculents. We calculate the $D_B$ of 197 grand-designs and 322 flocculents from SDSS DR18, using the samples of \citet{Buta..2015} and \citet{Sarkar..2023}. Our calculated median values of $D_B$ are $1.29^{+0.06}_{-0.04}$ and $1.38^{+0.05}_{-0.06}$ for the grand-designs and flocculents, respectively. In addition, a Kolmogorov-Smirnov (K-S) test rejects null hypothesis that these distributions are drawn from the same population. Finally, using a Random Forest (RF) model, we compare the effectiveness of $D_B$ in classifying spiral arm morphology, as compared to five other parameters viz. total atomic hydrogen HI mass $M_{HI}$, ratio of atomic hydrogen mass-to-blue luminosity $M_{HI} /L_B$, concentration index $C_i$, clumpiness $S$ and arm-contrast $C$. Our results indicate that $D_B$ has the highest feature index (30.8\%), followed by $C_i$ (26.0\%) and $M_{HI}$ (21.0\%). In fact, C, the metric routinely used to distinguish between the spiral morphologies has a feature importance of 8.3\%. Further, $D_B$ for grand-designs is found to anti-correlate with the central velocity dispersion with a correlation coefficient of -0.3 and $p \ll 0.05$. A high value of central velocity dispersion indicates a central Q-barrier, which favors the formation of grand designs according to the density wave theory. Thus, fractal dimension serves as a robust metric to distinguish between spiral morphologies and also links to the formation mechanism of spiral features.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2508.03814/./AlphaCen_stab.png', 'tmp_2508.03814/./AlphaCen_Orbit.png', 'tmp_2508.03814/./AcenF1000WwGaiaG0G5.png', 'tmp_2508.03814/./EpsMusF1000WwGaiaG9.png']
copying  tmp_2508.03814/./AlphaCen_stab.png to _build/html/
copying  tmp_2508.03814/./AlphaCen_Orbit.png to _build/html/
copying  tmp_2508.03814/./AcenF1000WwGaiaG0G5.png to _build/html/
copying  tmp_2508.03814/./EpsMusF1000WwGaiaG9.png to _build/html/
exported in  _build/html/2508.03814.md
    + _build/html/tmp_2508.03814/./AlphaCen_stab.png
    + _build/html/tmp_2508.03814/./AlphaCen_Orbit.png
    + _build/html/tmp_2508.03814/./AcenF1000WwGaiaG0G5.png
    + _build/html/tmp_2508.03814/./EpsMusF1000WwGaiaG9.png
found figures ['tmp_2508.04254/./FIG/Sed_acc_2.png', 'tmp_2508.04254/./FIG/Rim_22_radial_ts.png', 'tmp_2508.04254/./FIG/Rhotemp_MAG001.png', 'tmp_2508.04254/./FIG/Tempmag_part1.png']
copying  tmp_2508.04254/./FIG/Sed_acc_2.png to _build/html/
copying  tmp_2508.04254/./FIG/Rim_22_radial_ts.png to _build

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\teff}{\ensuremath{T_{\rm eff}}}$
$\newcommand{\rsun}{\ensuremath{ R_\Sun}}$
$\newcommand{\lsun}{\ensuremath{ L_\Sun}}$
$\newcommand{\mj}{\ensuremath{ M_{\rm Jup}}}$
$\newcommand{\rj}{\ensuremath{R_{\rm Jup}}}$
$\newcommand{\mearth}{M_\oplus}$
$\newcommand{\rearth}{R_\oplus}$
$\newcommand{\mum}{\mum}$
$\newcommand{◦ee}{^\circ}$
$\newcommand{\acenA}{\alpha~Cen~A\xspace}$
$\newcommand{\acen}{\alpha~Cen\xspace}$
$\newcommand{\acenAb}{\alpha~Cen~Ab\xspace}$
$\newcommand{\acenB}{\alpha~Cen~B\xspace}$
$\newcommand{\acenAB}{\alpha~Cen~AB\xspace}$
$\newcommand{\acenABC}{\alpha~Cen~ABC\xspace}$
$\newcommand{\emus}{\epsilon~Mus\xspace}$
$\newcommand{\sone}{S1\xspace}$
$\newcommand{\chas}[1]{\textcolor{orange}{#1}}$
$\newcommand{\aniket}[1]{\textcolor{blue}{#1}}$
$\newcommand{\pierre}[1]{\textcolor{magenta}{#1}}$</div>



<div id="title">

# Worlds Next Door: A Candidate Giant Planet Imaged in the Habitable Zone of $\acenA$.\ I. Observations, Orbital and Physical Properties, and Exozodi Upper Limits

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.03814-b31b1b.svg)](https://arxiv.org/abs/2508.03814)<mark>Appeared on: 2025-08-07</mark> -  _Accepted to ApJL. 34 pages, 22 figures, 10 tables_

</div>
<div id="authors">

C. Beichman, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** We report on coronagraphic observations of the nearest solar-type star, $\acenA$ , using the MIRI instrument on the $*James Webb*$ Space Telescope. The proximity of $\acen$ (1.33 pc) means that the star's habitable zone is spatially resolved at mid-infrared wavelengths, so sufficiently large planets or quantities of exozodiacal dust would be detectable via direct imaging. With three epochs of observation (August 2024, February 2025, and April 2025), we achieve a sensitivity sufficient to detect $T_{\rm eff}\approx$ 225--250 K (1--1.2 $R_{\rm Jup}$ ) planets between 1 $\arcsec$ --2 $\arcsec$ and exozodiacal dust emission at the level of $>$ 5--8 $\times$ the brightness of our own zodiacal cloud. The lack of exozodiacal dust emission sets an unprecedented limit of a few times the brightness of our own zodiacal cloud---a factor of $\gtrsim$ 5--10 more sensitive than measured toward any other stellar system to date. In August 2024, we detected a F $_\nu$ (15.5 $\mum$ ) = 3.5 mJy point source, called $\sone$ , at a separation of 1.5 $\arcsec$ from $\acenA$ at a contrast level of $5.5\times10^{-5}$ . Because the August 2024 epoch had only one successful observation at a single roll angle, it is not possible to unambiguously confirm $\sone$ as a bona fide planet. Our analysis confirms that $\sone$ is neither a background nor a foreground object. $\sone$ is not recovered in the February and April 2025 epochs. However, if $\sone$ is the counterpart of the object, $C1$ , seen by the VLT/NEAR program in 2019, we find that there is a 52 \% chance that the $S1+C1$ candidate was missed in both follow-up JWST/MIRI observations due to orbital motion. Incorporating constraints from the non-detections, we obtain families of dynamically stable orbits for $S1+C1$ with periods between 2--3 years. These suggest that the planet candidate is on an eccentric ( $e \approx 0.4$ ) orbit significantly inclined with respect to the $\acenAB$ orbital plane ( $i_{\rm mutual} \approx 50^\circ$ , prograde, or $\approx 130^\circ$ , retrograde). Based on the photometry and inferred orbital properties, the planet candidate could have a temperature of 225 K, a radius of $\approx$ 1--1.1 $\rj$ and a mass between 90--150 $\mearth$ , consistent with RV limits. This paper is first in a series of two papers: Paper II (Sanghi \& Beichman et al. 2025, in press) discusses the data reduction strategy and finds that $\sone$ is robust as a planet candidate, as opposed to an image or detector artifact.

</div>

<div id="div_fig1">

<img src="tmp_2508.03814/./AlphaCen_stab.png" alt="Fig19" width="100%"/>

**Figure 19. -** The planet candidate is initialized using the mean values from Table \ref{tab:orbit} with $K_{\rm RV}<3 {\rm m/s}$, where the color-coded points denote the planet candidate with a prograde (blue/cyan) or retrograde (red/magenta) orbit. The top panel shows the lifetime $t_{99}$ when $99\%$ of the test particles are unstable, while the bottom panel measures the fraction of stable particles for a given initial semi-major axis. The triangles denote upper limits for the stable fraction due to the finite number of trials (360) per semimajor axis. The light green region denotes the optimistic HZ, while the dark green represents the more conservative HZ.
  (*fig:PlanetStability*)

</div>
<div id="div_fig2">

<img src="tmp_2508.03814/./AlphaCen_Orbit.png" alt="Fig21" width="100%"/>

**Figure 21. -** *Left:* Orbit of $\acen$B relative to $\acenA$ showing one epoch of Hipparcos and two epochs of ALMA data. *Center:* zoom-in showing the ALMA data. *Top right:* The difference between the parallax effect as seen from Earth and from JWST at L2. *Bottom right:* The uncertainty in position of $\acen$ as a function of year before and after the addition of the new 2023 ALMA observations. (*fig:ALMA1*)

</div>
<div id="div_fig3">

<img src="tmp_2508.03814/./AcenF1000WwGaiaG0G5.png" alt="Fig12.1" width="50%"/><img src="tmp_2508.03814/./EpsMusF1000WwGaiaG9.png" alt="Fig12.2" width="50%"/>

**Figure 12. -** *Left:* F1000W image of $\acenA$B showing Gaia stars (green boxes) and MIRI detections (red boxes). The stars labeled $G0$ and $G5$ were used for target acquisition of $\acenA$. *Right:* similar F1000W image for $\emus$. The star labeled $G9$ was used for target acquisition of $\emus$.  (*fig:TAobs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.03814"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand{\mum}{\upmum}$
$\newcommand{\Omegas}{\mathit{\Omega}}$</div>



<div id="title">

# Multi dust species inner rim in magnetized protoplanetary disks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.04254-b31b1b.svg)](https://arxiv.org/abs/2508.04254)<mark>Appeared on: 2025-08-07</mark> -  _accepted at A&A_

</div>
<div id="authors">

<mark>M. Flock</mark>, et al. -- incl., <mark>M. Benisty</mark>, <mark>R. v. Boekel</mark>

</div>
<div id="abstract">

**Abstract:** The inner regions of protoplanetary disks, within ten astronomical units (au), are where terrestrial planets are born. Developing multi-physics models of this environment is crucial for understanding how planets form.By developing a new class of multi-dust radiative magnetized inner rim models and comparing them with recent near-infrared observational data, we can gain valuable insights into the conditions during planet formation. Our goal is twofold: to study the influence of highly refractory dust species on the inner rim shape and to determine how the magnetic field affects the inner disk structure. The resulting temperature and density structures are analyzed and compared to observations. The comparison focuses on a median SED of Herbig stars and interferometric constraints from the H, K, and N-band of three Herbig-type star-disk systems: \texttt{HD 100546} , \texttt{HD 163296} , and \texttt{HD 169142} . With the new models, we investigate 1) the influence of a large-scale magnetic field on the inner disk structure and 2) the effect of having the four most important dust species (corundum, iron, forsterite, and enstatite) shaping the rim, each with its sublimation temperatures.Further, we improve our model by using frequency-dependent irradiation and the effect of accretion heating. With the \texttt{Optool} package, we obtain frequency-dependent opacities for each dust grain family and calculate the corresponding temperature-dependent Planck and Rosseland opacities. When multiple dust species are considered, the dust's sublimation front, i.e., the inner rim, becomes smoother and radially more extended. Strongly magnetized disks show a substantial increase in the emission flux between the L and N-bands. Our results show that weakly magnetized disk models with large-scale vertical magnetic fields $\leq 0.3$ Gauss at 1 au best fit with NIR interferometric observations. Our model comparison supports the existence of moderate magnetic fields ( $\beta \ge 10^4$ ), which could drive a magnetic wind in the inner disk. Our results show that multi-dust models, including magnetic fields, still lack NIR emission, especially in the H-band. Half-light radii derived from H-band emission by near-infrared interferometry indicate that the missing flux originates within the inner rim, where even corundum grains sublimate. One potential solution might be a heated gas disk or evaporating objects like planetesimals close to the star.

</div>

<div id="div_fig1">

<img src="tmp_2508.04254/./FIG/Sed_acc_2.png" alt="Fig6.1" width="50%"/><img src="tmp_2508.04254/./FIG/Rim_22_radial_ts.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Top: SED for additional models: with accretion heating (red dashed), with only silicate grains (green dotted), and with only silicates with a constant and high ($T_{\rm ev}=1500K$) sublimation temperature \texttt{M001\_S\_TC}(blue dotted). The red dashed line was calculated based on model \texttt{M001}, including accretion heating in the \texttt{RADMC3D} calculation. Bottom: Radial intensity profile { along the major axis (Y=0)} for models \texttt{M001}, \texttt{M03},  \texttt{M001} with accretion heating in \texttt{RADMC3D} and the model \texttt{M001\_S\_TC}{at K-band (2.2 $\mum$)}. (*fig:sed_acc*)

</div>
<div id="div_fig2">

<img src="tmp_2508.04254/./FIG/Rhotemp_MAG001.png" alt="Fig12" width="100%"/>

**Figure 12. -** 2D temperature and gas density profile of model \texttt{M001}. The green dotted line shows plasma beta unity. {The black dotted line shows the MRI transition (900 K contour line)}. The circle indicates the location of the pressure maxima where { inward drifting pebbles} can be trapped. The dotted magenta line shows the sublimation of the corundum. The white solid line shows the water snowline. The blue line shows the location of maximal irradiation heating corresponding to the $\tau_\mathrm{r} = 1$ line. {Yellow and orange small dashes show the $\tau_\mathrm{z} = 1$ line at 2.2 and 10 $\mu$m (K-band and N-band). White separated small dashes show the $\tau_\mathrm{z} = 1$ for 1 mm emission.} (*fig:inittemp*)

</div>
<div id="div_fig3">

<img src="tmp_2508.04254/./FIG/Tempmag_part1.png" alt="Fig14" width="100%"/>

**Figure 14. -** 2D temperature profile with increasing magnetization from top to bottom showing models: \texttt{M001},\texttt{M01} and \texttt{M03}. The green dotted line shows the plasma beta unity line. The blue line shows the location of maximal irradiation heating corresponding to the $\tau_\mathrm{r} = 1$ line. Using the local Planck Opacity, the yellow dotted line shows the $\tau_\mathrm{z} = 1$ line. The white solid line shows the water snowline. (*fig:tempmag*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.04254"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]{\unskip\protect\href{https://orcid.org/#1}{\protect\includegraphics[width=8pt,clip]{logo_orcid}}}$
$\newcommand{\Lsun}{L_{\odot}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\Mearth}{M_{\oplus}}$
$\newcommand{\Rsun}{R_{\odot}}$
$\newcommand{\Mdot}{\dot{M}}$
$\newcommand{\Mdust}{{M_{\rm{dust}}}}$
$\newcommand{\Mdisk}{{M_{\rm{disk}}}}$
$\newcommand{\Av}{A_V}$
$\newcommand{\msunyr}{\rm{M_{\sun}   yr^{-1}}}$
$\newcommand{\Teff}{T_{\rm eff}}$
$\newcommand{\mic}{\mum}$
$\newcommand{\tstar}{T_{*}}$
$\newcommand{\rstar}{R_{*}}$
$\newcommand{\mstar}{M_{*}}$
$\newcommand{\rdisk}{r_{\rm{disk}}}$
$\newcommand{\mdisk}{M_{\rm{disk}}}$
$\newcommand{\hwall}{h_{\rm{wall}}}$
$\newcommand{\amaxbig}{a_{\rm{max,big}}}$
$\newcommand{\amaxsmall}{a_{\rm{max,small}}}$
$\newcommand{\brgamma}{Br\gamma}$
$\newcommand{\Lacc}{L_{\rm{acc}}}$
$\newcommand{\LaccL}{L_{\rm{acc}}/L_{*}}$
$\newcommand{\Lbrg}{L_{\rm{Br\gamma}}}$
$\newcommand{\Fbrg}{F_{\rm{Br\gamma}}}$
$\newcommand{\angstrom}{\mbox{\normalfontÅ}}$
$\newcommand{\Lstar}{L_{*}}$
$\newcommand{\LNIR}{L_{\rm{NIR}}}$
$\newcommand{\LIR}{L_{\rm{IR}}}$
$\newcommand{\vsini}{vsin(i)}$
$\newcommand{\Ri}{R_{i}}$
$\newcommand{\mdotmdisk}{\dot{M}--M_{\rm{disk}}}$
$\newcommand{\tdisk}{t_{\rm{disk}}}$
$\newcommand{\sg}[1]{\textcolor{red}{Sierra}: #1}$</div>



<div id="title">

# MINDS. A transition from $H_2$O to $C_2$$H_2$ dominated spectra with decreasing stellar luminosity

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.04692-b31b1b.svg)](https://arxiv.org/abs/2508.04692)<mark>Appeared on: 2025-08-07</mark> -  _Accepted for publication in A&A. 15 pages, 9 figures. ArXiv abstract is shortened_

</div>
<div id="authors">

S. L. Grant, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark>

</div>
<div id="abstract">

**Abstract:** The chemical composition of the inner regions of disks around young stars will largely determine the properties of planets forming in these regions. Many disk physical processes drive the disk chemical evolution, some of which depend on and/or correlate with the stellar properties. We aim to explore the connection between stellar properties and the inner disk chemistry in protoplanetary disks, as traced by mid-infrared spectroscopy. We use JWST-MIRI observations of a large, diverse sample of sources to explore trends between the carbon-bearing molecule $C_2$ $H_2$ and the oxygen-bearing molecule $H_2$ O. Additionally, we calculate the average spectrum for the T Tauri ( $\mstar$ $>$ 0.2 $\Msun$ ) and very low-mass star (VLMS, $\mstar$ $\leq$ 0.2 $\Msun$ ) samples from JWST-MIRI MRS data and use slab models to determine the properties of the average spectra in each subsample. We find a significant anti-correlation between the flux ratio of $C_2$ $H_2$ /$H_2$ O and the stellar luminosity. Disks around VLMS have significantly higher $F_{\rm{C_2H_2}}$ / $F_{\rm{H_2O}}$ flux ratios than their higher-mass counterparts, driven by the generally weak $H_2$ O and strong $C_2$ $H_2$ in disks around low-mass hosts. We also explore trends with the strength of the 10 $\mu$ m silicate feature, the stellar accretion rate, and the disk dust mass, all of which show correlations with $F_{\rm{C_2H_2}}$ / $F_{\rm{H_2O}}$ , which may be related to processes driving the carbon-enrichment in disks around VLMS, but also have degeneracies with system properties (i.e., the $\mstar$ -- $\Mdot$ and $\mstar$ -- $\mdisk$ relationships). Slab model fits to the average spectra show that $H_2$ O emission in the VLMS sample is quite similar in temperature and column density to a warm ( $\sim$ 600 K) $H_2$ O component in the T Tauri spectrum, indicating that the high C/O gas phase ratio in these disks is not due to oxygen depletion alone. Instead, the presence of many hydrocarbons, including some with high column densities, suggests that carbon enhancement in the disks around VLMS is taking place. The observed differences in the inner disk chemistry as a function of host properties are likely to be accounted for by differences in the disk temperatures, stellar radiation field, and the evolution of dust grains.

</div>

<div id="div_fig1">

<img src="tmp_2508.04692/./C2H2_H2O_10mic_Mdot_Mdust.png" alt="Fig3" width="100%"/>

**Figure 3. -** Left: $F_{\rm{C_2H_2}}$/$F_{\rm{H_2O}}$ as a function of the strength of the 10 $\mic$ silicate feature (stronger silicate features have higher values). For four of the VLMS, there is either no 10 $\mic$ emission or the emission is coming at least partially from $C_2$$H_4$, therefore we adopt feature strength of 1 for these sources (denoted by open points). Two outliers at a 10 $\mic$ band strength of 3 and 3.8 are the transitional disks LkCa 15 and PDS 70 and have been removed for clarity. Middle: The relationship between $F_{\rm{C_2H_2}}$/$F_{\rm{H_2O}}$ and $\Mdot$. Right: The relationship between $F_{\rm{C_2H_2}}$/$F_{\rm{H_2O}}$ and $\Mdust$. The PCCs and $p-$values can be found for each panel. All of the relationships are statistically significant ($p-$value$<$0.05); however the correlations are not as strong as the $F_{\rm{C_2H_2}}$/$F_{\rm{H_2O}}$ vs. stellar luminosity relationship. Rotated triangular markers for the VLMS sample indicate lower limits on $F_{\rm{C_2H_2}}$/$F_{\rm{H_2O}}$ and upper limits on $\Mdust$. Error bars are smaller than the points for most targets.  (*fig: C2H2 H2O Mstar Mdot Mdust*)

</div>
<div id="div_fig2">

<img src="tmp_2508.04692/./VLMS_chi2.png" alt="Fig9" width="100%"/>

**Figure 9. -** Same as Figure \ref{fig: chi2 ttauri}, but now for the molecules in the average VLMS spectrum. (*fig: chi2 vlms*)

</div>
<div id="div_fig3">

<img src="tmp_2508.04692/./molecules_T_N_R_errors_limits.png" alt="Fig5" width="100%"/>

**Figure 5. -** The best-fit slab model parameters for the average VLMS spectrum (left circles) and T Tauri spectrum (right squares) for the different molecules. The $H_2$O component for the T Tauri spectrum is the intermediate ($T\sim600$) component. Temperature and column density are shown on the top on the left and right, respectively. The equivalent emitting radius is shown on the bottom left and that radius normalized to the $H_2$O snowline for each subsample is shown on the bottom right. Only the optically thick $C_2$$H_2$ component is shown for the VLMS in the radii plots, as the radius is unconstrained in the optically thin case. Error bars are determined from the 1$\sigma$ confidence intervals and in some cases are degenerate between parameters. For example, in the case of optically thin emission, the column density and equivalent emitting radii are degenerate. See Figures \ref{fig: chi2 ttauri} and \ref{fig: chi2 vlms} for the $\chi^2$ maps for the T Tauri and VLMS fits, respectively. (*fig: avg params*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.04692"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

138  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
